In [1]:
# Set the base URL
BASE_URL="http://localhost:8080"

In [2]:
import json
from requests import get, put, post, delete

In [3]:
from urllib.parse import urlencode

mit_code = "def SIDARTHE_model(y, t, alpha, beta, gamma, delta, epsilon, mu, zeta, lamb, eta, rho, theta, kappa, nu, xi, sigma, tau): S, I, D, A, R, T, H, E = y dSdt = -S*(alpha(t)*I + beta(t)*D + gamma(t)*A + delta(t)*R) dIdt = S*(alpha(t)*I + beta(t)*D + gamma(t)*A + delta(t)*R) - (epsilon(t) + zeta(t) + lamb(t))*I dDdt = epsilon(t)*I - (eta(t) + rho(t))*D dAdt = zeta(t)*I - (theta(t) + mu(t) + kappa(t))*A dRdt = eta(t)*D + theta(t)*A - (nu(t) + xi(t))*R dTdt = mu(t)*A + nu(t)*R - (sigma(t) + tau(t))*T dHdt = lamb(t)*I + rho(t)*D + kappa(t)*A + xi(t)*R + sigma(t)*T dEdt = tau(t)*T return dSdt, dIdt, dDdt, dAdt, dRdt, dTdt, dHdt, dEdt"

mit_json = { "code":mit_code ,"gpt_key":"insert_here"}

encoded_query = urlencode(mit_json)

MIT_states_url = "http://100.26.10.46/petri/get_places/" + "?" + encoded_query

MIT_transitions_url = "http://100.26.10.46/petri/get_parameters/" + "?" + encoded_query

mit_sp = post(MIT_states_url).json()

mit_trans = post(MIT_transitions_url).json()

print(mit_sp)

print(mit_trans)

['S', ' I', ' D', ' A', ' R', ' T', ' H', ' E']
['alpha', ' beta', ' gamma', ' delta', ' epsilon', ' mu', ' zeta', ' lamb', ' eta', ' rho', ' theta', ' kappa', ' nu', ' xi', ' sigma', ' tau']


In [54]:
import acsets, petris

mit_sir = petris.Petri()
mit_sir.add_species(len(mit_sp))
trans = petris.Transition
mit_sir.add_parts(trans, len(mit_trans))

i = 0
for i, tran in enumerate(mit_trans): 
    mit_sir.set_subpart(i, petris.attr_tname, mit_trans[i].strip())
    mit_sir.set_subpart(i, petris.attr_tuid, i)

j = 0
for j, spec in enumerate(mit_sp): 
    mit_sir.set_subpart(j, petris.attr_sname, mit_sp[j].strip())
    mit_sir.set_subpart(j, petris.attr_suid, i+j)

mit_serialized = mit_sir.write_json()
with open("mit_py-ascet.json", "w") as outfile:
    outfile.write(mit_serialized)
p = open('mit_py-ascet.json')
p_data = json.load(p)

f = open('test_union_metadata.json')
a_data = json.load(f)

temp_json = {}

for m, entry in enumerate(a_data):
    for n, e2 in enumerate(p_data["S"]):
        if e2["sname"] == entry:
            uid = e2["uid"]
            temp_json.update({uid:a_data[entry]})

with open("test_union_metadata_uid.json", "w") as outfile:
    json.dump(temp_json, outfile)
            
print(temp_json)



#print(mit_serialized)
#print(data)

{15: 'this is metadata for S', 16: 'This is metadata for I'}


In [62]:
MODEL_ID = 2835
opos = get(f"{BASE_URL}/models/{MODEL_ID}/named_opos").json()
trimmed_opos = []
for ent in opos: 
    trimmed_opos.append(ent.replace("d", "").replace("t", ""))

print(trimmed_opos)

['S', 'I', 'D', 'A', 'R', 'T', 'H', 'E']


In [63]:
opis = get(f"{BASE_URL}/models/{MODEL_ID}/named_opis").json()
opis = opis[3:]
print(opis)

['gamma', 'delta', 'epsilon', 'mu', 'zeta', 'lamb', 'eta', 'rho', 'theta', 'kappa', 'nu', 'xi', 'sigma', 'tau']


In [44]:
edges = [([0,1],[1,1]),([1],[2])]

In [3]:
import acsets, petris

sir = petris.Petri()
sir.add_species(len(trimmed_opos))
trans = petris.Transition
sir.add_parts(trans, len(opis))

i=0
for i, tran in enumerate(opis): 
    sir.set_subpart(i, petris.attr_tname, opis[i])
    sir.set_subpart(i, petris.attr_tuid, i)

j=0
for j, spec in enumerate(trimmed_opos): 
    sir.set_subpart(j, petris.attr_sname, trimmed_opos[j])
    sir.set_subpart(j, petris.attr_suid, i+j)

'''for t, ([ins, outs]) in enumerate(edges):
    for s in ins:
        arc = sir.add_part(petris.Input)
        sir.set_subpart(arc, petris.hom_it, t)
        sir.set_subpart(arc, petris.hom_is, s)
    for r in outs:
        arc = sir.add_part(petris.Output)
        sir.set_subpart(arc, petris.hom_ot, t)
        sir.set_subpart(arc, petris.hom_os, r)'''

serialized = sir.write_json()

with open("____seird_sample.json", "w") as outfile:
    outfile.write(serialized)
f = open('____seird_sample.json')
data = json.load(f)

print(serialized)
print(data)

NameError: name 'trimmed_opos' is not defined

In [1]:
import acsets, petris

sp = ['s', 'i', 'r']

t = ['beta', 'gamma']

sir = petris.Petri()
s,i,r = sir.add_species(3)
inf,rec = sir.add_transitions([([0,1],[1,1]),([1],[2])])

for k, tran in enumerate(t): 
    sir.set_subpart(k, petris.attr_tname, t[k])
    sir.set_subpart(k, petris.attr_tuid, k)

for j, spec in enumerate(sp): 
    sir.set_subpart(j, petris.attr_sname, sp[j])
    sir.set_subpart(j, petris.attr_suid, j)

serialized = sir.write_json()

print(serialized)

{"S": [{"sname": "s", "uid": 0}, {"sname": "i", "uid": 1}, {"sname": "r", "uid": 2}], "T": [{"tname": "beta", "uid": 0}, {"tname": "gamma", "uid": 1}], "I": [{"it": 1, "is": 1}, {"it": 1, "is": 2}, {"it": 2, "is": 2}], "O": [{"ot": 1, "os": 2}, {"ot": 1, "os": 2}, {"ot": 2, "os": 3}]}


In [2]:
import acsets, petris
import json

f = open('model_pn_SIDARTHE_V.json')
eq_data = json.load(f)

species = eq_data["species_latex"]
print(species)
transitions = eq_data["rates_latex"]
print(transitions)
edges = eq_data["edges"]
print(edges)

sir = petris.Petri()
sir.add_species(len(species))
trans = petris.Transition
sir.add_parts(trans, len(transitions))

i=0
for i, tran in enumerate(transitions): 
    sir.set_subpart(i, petris.attr_tname, transitions[i])
    sir.set_subpart(i, petris.attr_tuid, i)

j=0
for j, spec in enumerate(species): 
    sir.set_subpart(j, petris.attr_sname, species[j])
    sir.set_subpart(j, petris.attr_suid, i+j)

for t, ([ins, outs]) in enumerate(edges):
    for s in ins:
        arc = sir.add_part(petris.Input)
        sir.set_subpart(arc, petris.hom_it, t)
        sir.set_subpart(arc, petris.hom_is, s)
    for r in outs:
        arc = sir.add_part(petris.Output)
        sir.set_subpart(arc, petris.hom_ot, t)
        sir.set_subpart(arc, petris.hom_os, r)

serialized = sir.write_json()

print(serialized)

with open("SKEMA_SIDARTHE_PN_V.json", "w") as outfile:
    outfile.write(serialized)


['D', 'V', 'A', 'I', 'R', 'T', 'H', 'E', 'S']
['α', 'β', 'γ', 'δ', 'ϕ', 'ϵ', 'ζ', 'λ', 'η', 'ρ', 'θ', 'μ', 'κ', 'ν', 'ξ', 'τ_{1}', 'σ', 'τ_{2}']
[[[8, 3], [3, 3]], [[8, 0], [0, 3]], [[8, 2], [2, 3]], [[8, 4], [4, 3]], [[8], [1]], [[3], [0]], [[3], [2]], [[3], [6]], [[0], [4]], [[0], [6]], [[2], [4]], [[2], [5]], [[2], [6]], [[4], [5]], [[4], [6]], [[4], [7]], [[5], [6]], [[5], [7]]]
{"S": [{"sname": "D", "uid": 17}, {"sname": "V", "uid": 18}, {"sname": "A", "uid": 19}, {"sname": "I", "uid": 20}, {"sname": "R", "uid": 21}, {"sname": "T", "uid": 22}, {"sname": "H", "uid": 23}, {"sname": "E", "uid": 24}, {"sname": "S", "uid": 25}], "T": [{"tname": "\u03b1", "uid": 0}, {"tname": "\u03b2", "uid": 1}, {"tname": "\u03b3", "uid": 2}, {"tname": "\u03b4", "uid": 3}, {"tname": "\u03d5", "uid": 4}, {"tname": "\u03f5", "uid": 5}, {"tname": "\u03b6", "uid": 6}, {"tname": "\u03bb", "uid": 7}, {"tname": "\u03b7", "uid": 8}, {"tname": "\u03c1", "uid": 9}, {"tname": "\u03b8", "uid": 10}, {"tname": "\u03

[([], []), ([], []), ([], [])]